# Visualizing hypergraphs

Visualizing hypergraphs, just like pairwise networks, is a hard task and no algorithm can work nicely for any given input structure. Here, we show how to visualize some toy structures using the visualization function contained in the ```drawing``` module that is often inspired by [networkx](https://networkx.org/documentation/stable/reference/drawing.html) and relies on [matplotlib](https://matplotlib.org/).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xgi

## Basics

Les us first create a small toy hypergraph containing edges of different sizes.

In [ ]:
H = xgi.Hypergraph()
H.add_edges_from(
    [[1, 2, 3], [3, 4, 5], [3, 6], [6, 7, 8, 9], [1, 4, 10, 11, 12], [1, 4]]
)

It can be quickly visualized simply with

In [ ]:
xgi.draw(H)

By default, the **layout** is computed by ```xgi.barycenter_spring_layout```. For a bit more control, 
we can compute a layout externally (that we fix with a random seed), so that we can reuse it:

In [ ]:
pos = xgi.barycenter_spring_layout(H, seed=1)

fig, ax = plt.subplots()
xgi.draw(H, pos=pos, ax=ax)

**Labels** can be added to the nodes and hyperedges with arguments `node_labels` and `hyperedge_labels`. If ```True```, the IDs are shown. To display user-defined labels, pass a dictionary that contains (id: label). Additional keywords related to the font can be passed as well:

In [ ]:
xgi.draw(H, pos, node_labels=True, node_size=15, hyperedge_labels=True)
plt.show()

Note that by default, the nodes are drawn too small (size 7) to display the labels nicely. For better visuals, increase the node size to at least 15 when displaying node labels.

## Other types of visualizations 

Another common way of visualizing hypergraph is with **convex hulls** as hyperedges. This can be done simply by setting `hull=True`:

In [ ]:
fig, ax = plt.subplots()
xgi.draw(H, pos=pos, ax=ax, hull=True)

plt.show()

XGI also offer a function to visualize a hypergraph as a **multilayer**, where each layer contains hyperedges of a given order:

In [ ]:
ax3 = plt.axes(projection="3d")  # requires a 3d axis
xgi.draw_multilayer(H, ax=ax3, node_fc=H.nodes.degree)

plt.show()

## Colors and sizes

The drawing functions offer great flexibility in choosing the width, size, and color of the elements that are drawn. 

By default, hyperedges are colored according to their order. Hyperedge and node colors can be set manually to a single color, a list of colors, or a by an array/dict/Stat of numerical values. 

In the latter case, numerical values are mapped to colors via a colormap which can be changed manually (see [colormaps](https://matplotlib.org/stable/tutorials/colors/colormaps.html) for details):

First, let's use single values:

In [ ]:
fig, ax = plt.subplots()

xgi.draw(
    H,
    pos=pos,
    ax=ax,
    node_fc="k",
    node_ec="beige",
    node_shape="s",
    node_size=10,
    node_lw=2,
    edge_fc="salmon",
    dyad_color="grey",
    dyad_lw=3,
)

plt.show()

Now we can use statistic to set the colors and sizes, and change the colormaps:

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2.5))

ax, collections = xgi.draw(
    H,
    pos=pos,
    node_fc=H.nodes.degree,
    edge_fc=H.edges.size,
    edge_fc_cmap="viridis",
    node_fc_cmap="mako_r",
)

node_col, _, edge_col = collections

plt.colorbar(node_col, label="Node degree")
plt.colorbar(edge_col, label="Edge size")

plt.show()

## Layouts

Other layout algorithms are available:
* ```random_layout```: position nodes uniformly at random in the unit square.
* ```pairwise_spring_layout```: to position the nodes using the Fruchterman-Reingold force-directed algorithm on the projected graph. 
* ```barycenter_spring_layout```: position the nodes using the Fruchterman-Reingold force-directed algorithm using an augmented version of the the graph projection of the hypergraph, where _phantom nodes_ (that we call barycenters) are created for each edge of order $d>1$ (composed of more than two nodes). Weights are then assigned to all hyperedges of order 1 (links) and to all connections to phantom nodes within each hyperedge to keep them together. Weights scale with the size of the hyperedges.
* ```weighted_barycenter_spring_layout```: same as ```barycenter_spring_layout```, but here the weighted version of the Fruchterman-Reingold force-directed algorithm is used.
* ```barycenter_kamada_kawai```: position nodes using Kamada-Kawai path-length cost-function using an augmented version of the the graph projection of the hypergraph.
* ```circular_layout```: position nodes on a circle.
* ```spiral_layout```: position nodes in a spiral layout.

Each layout returns a dictionary that maps nodes ID into (x, y) coordinates.

For example:

In [ ]:
plt.figure(figsize=(10, 4))

ax = plt.subplot(1, 2, 1)
pos_circular = xgi.circular_layout(H)
xgi.draw(H, pos=pos_circular, ax=ax)

ax = plt.subplot(1, 2, 2)
pos_spiral = xgi.spiral_layout(H)
xgi.draw(H, pos=pos_spiral, ax=ax)

## Simplicial complexes

Simplicial complexes can be visualized using the same functions as for Hypergraphs.

**Technical note:**
By definition, a simplicial complex object contains all sub-simplices. This would make the visualization heavy since all sub-simplices contained in a maximal simplex would overlap. The automatic solution for this, implemented by default in all the layouts, is to convert the simplicial complex into a hypergraph composed solely by its maximal simplices.

**Visual note:**
To visually distinguish simplicial complexes from hypergraphs, the ```draw``` function will also show all links contained in each maximal simplices (while omitting simplices of intermediate orders). 

In [ ]:
SC = xgi.SimplicialComplex()
SC.add_simplices_from([[3, 4, 5], [3, 6], [6, 7, 8, 9], [1, 4, 10, 11, 12], [1, 4]])

In [ ]:
pos = xgi.barycenter_spring_layout(SC, seed=1)

In [ ]:
xgi.draw(SC, pos=pos)

plt.show()

## Rotating a hypergraph

For some hypergraphs, it can be helpful to rotate the positions of the nodes relative to the principal axis. We can do this by generating node positions with any of the functions previously described and then using the function `pca_transform()`. For example:

In [ ]:
pos = xgi.barycenter_spring_layout(H, seed=1)

transformed_pos = xgi.pca_transform(pos)
xgi.draw(H, transformed_pos)

plt.show()

We can also rotate the node positions relative to the principal axis:

In [ ]:
# rotation in degrees
transformed_pos = xgi.pca_transform(pos, 30)
xgi.draw(H, transformed_pos)

plt.show()

## Larger example: generative model
We generate and visualize a [random hypergraph](https://doi.org/10.1093/comnet/cnx001).

In [ ]:
n = 100
is_connected = False
while not is_connected:
    H = xgi.fast_random_hypergraph(n, [0.015, 0.0001, 0.000005])
    is_connected = xgi.is_connected(H)
pos = xgi.barycenter_spring_layout(H, seed=0)

Since there are more nodes we reduce the ```node_size```.

In [ ]:
plt.figure(figsize=(10, 10))
ax = plt.subplot(111)
xgi.draw(H, pos=pos, ax=ax)

We can even size/color the nodes and edges by NodeStats or EdgeStats (e.g., degree, centrality, size, etc.)!

In [ ]:
plt.figure(figsize=(10, 10))
ax = plt.subplot(111)
xgi.draw(
    H,
    pos,
    node_size=H.nodes.degree,
    node_lw=H.nodes.average_neighbor_degree,
    node_fc=H.nodes.degree,
    ax=ax,
)

## Degree distribution
Using its simplest (higher-order) definition, the degree is the number of hyperedges (of any size) incident on a node.

In [ ]:
centers, heights = xgi.degree_histogram(H)

plt.figure(figsize=(12, 4))
ax = plt.subplot(111)

ax.bar(centers, heights)
ax.set_ylabel("Count")
ax.set_xlabel("Degree")
ax.set_xticks(np.arange(1, max(centers) + 1, step=1));

In [ ]:
plt.close("all")

## Challenge

* Generate a 3-uniform HPPM hypergraph of size 100 with a mean degree of 10 and an imbalance parameter of 0.95. Position the node according to a pairwise spring layout and with the principal axis horizontal. (1) Color the nodes according to the clustering coefficient. (2) Make the node markers square.